In [ ]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, precision_recall_curve
from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.over_sampling import SMOTE
import gc
import warnings
warnings.filterwarnings('ignore')

# Step 1: Load the Data
# Assume a single CSV file for the credit card dataset
data = pd.read_csv(r"/content/creditcard.csv")  # Replace with your dataset path

# Check for NaN in 'Class' and handle them
print(f"Number of NaN values in 'Class': {data['Class'].isna().sum()}")
data = data.dropna(subset=['Class'])  # Drop rows where 'Class' is NaN
data['Class'] = data['Class'].astype(int)  # Ensure 'Class' is integer (0 or 1)
print(f"Dataset shape after dropping NaN in 'Class': {data.shape}")

# Split into train and test (80-20 split)
train, test = train_test_split(data, test_size=0.2, random_state=42, stratify=data['Class'])
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

# Clean up memory
del data
gc.collect()

print(f"Train shape: {train.shape}, Test shape: {test.shape}")

# Step 2: Preprocessing
# Remove features with only one unique value
one_unique_col = [col for col in train.columns if train[col].nunique() == 1]
train.drop(columns=one_unique_col, inplace=True)
test.drop(columns=one_unique_col, inplace=True)

# Define categorical columns (adjust based on dataset)
base_categorical_cols = ['card_type', 'email_domain'] if 'card_type' in train.columns else []
m_cols = [f'M{i}' for i in range(1, 10) if f'M{i}' in train.columns]
categorical_cols = base_categorical_cols + m_cols
categorical_cols = [col for col in categorical_cols if col in train.columns]

# Handle missing values
numerical_cols = train.select_dtypes(include=['float64', 'int64']).columns.tolist()
numerical_cols = [col for col in numerical_cols if col not in ['Class', 'TransactionID']]
for col in numerical_cols:
    train[col].fillna(train[col].median(), inplace=True)
    test[col].fillna(test[col].median(), inplace=True)

for col in categorical_cols:
    train[col] = train[col].fillna('missing').astype(str)
    test[col] = test[col].fillna('missing').astype(str)

# Remove outliers (adjust based on dataset)
if 'Amount' in train.columns:
    train.drop(train[train['Amount'] > 10000].index, inplace=True)

# Step 3: Feature Engineering
# Time-based features
if 'Time' in train.columns:
    train['day'] = ((train['Time'] // (3600 * 24) - 1) % 7) + 1
    test['day'] = ((test['Time'] // (3600 * 24) - 1) % 7) + 1
    train['hour'] = ((train['Time'] // 3600) % 24) + 1
    test['hour'] = ((test['Time'] // 3600) % 24) + 1

def new_hr_feature(hr):
    if hr >= 7 and hr < 10:
        return "highwarningsign"
    elif hr >= 14 and hr < 16:
        return "lowestwarningsign"
    elif (hr >= 4 and hr < 7) or (hr >= 10 and hr < 14):
        return "mediumwarningsign"
    else:
        return "lowwarningsign"

if 'hour' in train.columns:
    train['hour_warning'] = train['hour'].apply(new_hr_feature)
    test['hour_warning'] = test['hour'].apply(new_hr_feature)

# Log transform Amount
if 'Amount' in train.columns:
    train['LogTransactionAmt'] = np.log(train['Amount'] + 1e-9)
    test['LogTransactionAmt'] = np.log(test['Amount'] + 1e-9)

# New feature for card3 (if exists)
if 'card3' in train.columns:
    def new_card3(row):
        try:
            val = float(row)
            return 'Positive' if val > 160 else 'Negative'
        except:
            return 'missing'
    train['new_card3'] = train['card3'].apply(new_card3)
    test['new_card3'] = test['card3'].apply(new_card3)

# Simplify card6 (if exists)
if 'card6' in train.columns:
    def replacetodebit(row):
        if row in ['debit or credit', 'charge card']:
            return 'debit'
        return row
    train['card6'] = train['card6'].apply(replacetodebit)
    test['card6'] = test['card6'].apply(replacetodebit)

# Group email domains (if exists)
if 'P_emaildomain' in train.columns:
    for dataset in [train, test]:
        dataset.loc[dataset['P_emaildomain'].isin(['gmail.com', 'gmail']), 'P_emaildomain'] = 'Google'
        dataset.loc[dataset['P_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx', 'yahoo.co.uk',
                                                  'yahoo.co.jp', 'yahoo.de', 'yahoo.fr', 'yahoo.es']),
                    'P_emaildomain'] = 'Yahoo'
        dataset.loc[dataset['P_emaildomain'].isin(['hotmail.com', 'outlook.com', 'msn.com',
                                                  'live.com.mx', 'hotmail.es', 'hotmail.co.uk',
                                                  'hotmail.de', 'outlook.es', 'live.com',
                                                  'live.fr', 'hotmail.fr']),
                    'P_emaildomain'] = 'Microsoft'
        dataset.loc[dataset['P_emaildomain'].isin(dataset['P_emaildomain'].value_counts()[
                    dataset['P_emaildomain'].value_counts() <= 500].index),
                    'P_emaildomain'] = 'Others'
        dataset['P_emaildomain'].fillna('NoInf', inplace=True)
        if 'R_emaildomain' in dataset.columns:
            dataset.loc[dataset['R_emaildomain'].isin(['gmail.com', 'gmail']), 'R_emaildomain'] = 'Google'
            dataset.loc[dataset['R_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx', 'yahoo.co.uk',
                                                      'yahoo.co.jp', 'yahoo.de', 'yahoo.fr', 'yahoo.es']),
                        'R_emaildomain'] = 'Yahoo'
            dataset.loc[dataset['R_emaildomain'].isin(['hotmail.com', 'outlook.com', 'msn.com',
                                                      'live.com.mx', 'hotmail.es', 'hotmail.co.uk',
                                                      'hotmail.de', 'outlook.es', 'live.com',
                                                      'live.fr', 'hotmail.fr']),
                        'R_emaildomain'] = 'Microsoft'
            dataset.loc[dataset['R_emaildomain'].isin(dataset['R_emaildomain'].value_counts()[
                        dataset['R_emaildomain'].value_counts() <= 300].index),
                        'R_emaildomain'] = 'Others'
            dataset['R_emaildomain'].fillna('NoInf', inplace=True)

# Transaction frequency
if 'card1' in train.columns:
    train['card1_freq'] = train.groupby('card1')['card1'].transform('count')
    test['card1_freq'] = test.groupby('card1')['card1'].transform('count')

if 'addr1' in train.columns:
    train['addr1_freq'] = train.groupby('addr1')['addr1'].transform('count')
    test['addr1_freq'] = test.groupby('addr1')['addr1'].transform('count')

# Interaction feature
if 'card1_freq' in train.columns and 'Amount' in train.columns:
    train['amt_per_card1'] = train['Amount'] / (train['card1_freq'] + 1e-9)
    test['amt_per_card1'] = test['Amount'] / (test['card1_freq'] + 1e-9)

# Step 4: Feature Preparation
# Select top features
numerical_cols = ['Amount', 'LogTransactionAmt', 'card1_freq', 'addr1_freq', 'amt_per_card1'] if 'card1_freq' in train.columns else ['Amount', 'LogTransactionAmt']
numerical_cols += [col for col in train.columns if col.startswith('V')]
numerical_cols = [col for col in numerical_cols if col in train.columns]

categorical_cols = ['ProductCD', 'card6', 'new_card3', 'P_emaildomain', 'R_emaildomain', 'hour_warning']
categorical_cols = [col for col in categorical_cols if col in train.columns]

# Handle categorical encoding
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    train[col] = train[col].astype(str)
    train_unique_values = train[col].unique()
    le.fit(train_unique_values)
    train[col] = le.transform(train[col])
    test[col] = test[col].astype(str)
    test[col] = test[col].apply(lambda x: x if x in train_unique_values else 'unknown')
    if 'unknown' not in le.classes_:
        le_classes = list(le.classes_)
        le_classes.append('unknown')
        le.classes_ = np.array(le_classes)
    test[col] = le.transform(test[col])
    label_encoders[col] = le

# Combine features
features = numerical_cols + categorical_cols
X = train[features]
y = train['Class']
X_test = test[features]

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# Step 5: Handle Class Imbalance with SMOTE
smote = SMOTE(random_state=42, sampling_strategy=0.5)
X_train, y_train = smote.fit_resample(X_scaled, y)

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

# Step 6: Hyperparameter Tuning with GridSearchCV
param_grid = {
    'max_depth': [5, 7, 9],
    'min_samples_split': [5, 10, 20],
    'min_samples_leaf': [2, 5],
    'class_weight': ['balanced'],
    'ccp_alpha': [0.01, 0.05, 0.1]
}
dt = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(dt, param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best model
best_dt = grid_search.best_estimator_
print(f"Best Parameters: {grid_search.best_params_}")

# Step 7: Adaptive Thresholding
val_probs = best_dt.predict_proba(X_val)[:, 1]
precisions, recalls, thresholds = precision_recall_curve(y_val, val_probs)
f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-9)
optimal_threshold = thresholds[np.argmax(f1_scores)]
print(f"Optimal Threshold: {optimal_threshold:.4f}")

# Step 8: Evaluate Model
train_probs = best_dt.predict_proba(X_train)[:, 1]
val_probs = best_dt.predict_proba(X_val)[:, 1]

train_pred = (train_probs >= optimal_threshold).astype(int)
val_pred = (val_probs >= optimal_threshold).astype(int)

# Calculate metrics
train_precision = precision_score(y_train, train_pred)
train_recall = recall_score(y_train, train_pred)
train_f1 = f1_score(y_train, train_pred)
train_auc = roc_auc_score(y_train, train_probs)
train_accuracy = accuracy_score(y_train, train_pred)

val_precision = precision_score(y_val, val_pred)
val_recall = recall_score(y_val, val_pred)
val_f1 = f1_score(y_val, val_pred)
val_auc = roc_auc_score(y_val, val_probs)
val_accuracy = accuracy_score(y_val, val_pred)

# Print train and validation metrics
print("\nTrain Metrics:")
print(f"Train Precision: {train_precision:.4f}")
print(f"Train Recall: {train_recall:.4f}")
print(f"Train F1: {train_f1:.4f}")
print(f"Train AUC: {train_auc:.4f}")
print(f"Train Accuracy: {train_accuracy:.4f}")

print("\nTest (Validation) Metrics:")
print(f"Test Precision: {val_precision:.4f}")
print(f"Test Recall: {val_recall:.4f}")
print(f"Test F1: {val_f1:.4f}")
print(f"Test AUC: {val_auc:.4f}")
print(f"Test Accuracy: {val_accuracy:.4f}")

# Feature Importance
feature_importance = pd.DataFrame({
    'Feature': features,
    'Importance': best_dt.feature_importances_
}).sort_values(by='Importance', ascending=False)
print("\nFeature Importance:")
print(feature_importance)

Number of NaN values in 'Class': 1
Dataset shape after dropping NaN in 'Class': (15935, 31)
Train shape: (12748, 31), Test shape: (3187, 31)
Best Parameters: {'ccp_alpha': 0.01, 'class_weight': 'balanced', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 5}
Optimal Threshold: 0.9041

Train Metrics:
Train Precision: 0.9884
Train Recall: 0.9894
Train F1: 0.9889
Train AUC: 0.9932
Train Accuracy: 0.9926

Test (Validation) Metrics:
Test Precision: 0.9898
Test Recall: 0.9929
Test F1: 0.9913
Test AUC: 0.9949
Test Accuracy: 0.9942

Feature Importance:
              Feature  Importance
13                V12    0.953862
5                  V4    0.024124
15                V14    0.022014
2                  V1    0.000000
3                  V2    0.000000
4                  V3    0.000000
1   LogTransactionAmt    0.000000
6                  V5    0.000000
7                  V6    0.000000
9                  V8    0.000000
8                  V7    0.000000
10                 V9    0.0000